Walmart Weekly Sales Prediction

Importing Data

In [0]:
from google.colab import files
uploaded = files.upload()

Saving features.csv to features.csv
Saving sampleSubmission.csv to sampleSubmission.csv
Saving stores.csv to stores.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [0]:
import pandas as pd
import io
features = pd.read_csv(io.BytesIO(uploaded['features.csv']))
test = pd.read_csv(io.BytesIO(uploaded['test.csv']))
stores = pd.read_csv(io.BytesIO(uploaded['stores.csv']))
train = pd.read_csv(io.BytesIO(uploaded['train.csv']))

In [0]:
features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [0]:
#features['Tot_MDowns'] = features['MarkDown1'] + features['MarkDown2'] + features['MarkDown3'] + features['MarkDown4'] + features['MarkDown5']

In [0]:
features.tail()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Tot_MDowns
8185,45,2013-06-28,76.05,3.639,4842.29,975.03,3.00,2449.97,3169.69,NaN,NaN,False,11439.98
8186,45,2013-07-05,77.50,3.614,9090.48,2268.58,582.74,5797.47,1514.93,NaN,NaN,False,19254.20
8187,45,2013-07-12,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,NaN,NaN,False,8598.17
8188,45,2013-07-19,82.84,3.737,2961.49,1047.07,204.19,363.00,1059.46,NaN,NaN,False,5635.21
8189,45,2013-07-26,76.06,3.804,212.02,851.73,2.06,10.88,1864.57,NaN,NaN,False,2941.26


Dummify IsHoliday

In [0]:
features['IsHoliday'] = (features['IsHoliday'] == True ).astype(int)

In [0]:
features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Tot_MDowns
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,0,NaN
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,1,NaN
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,0,NaN
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,0,NaN
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,0,NaN


Working out the date formats

In [0]:
features['Date'] = pd.to_datetime(features['Date'])

In [0]:
features['Month'] = features['Date'].dt.month
features['Day'] = features['Date'].dt.day

In [0]:
features['Year'] = features['Date'].dt.year

In [0]:
features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Month,Day,Year
0,1,2010-02-05,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,0,2,5,2010
1,1,2010-02-12,38.51,2.548,0.0,0.0,0.0,0.0,0.0,211.242170,8.106,1,2,12,2010
2,1,2010-02-19,39.93,2.514,0.0,0.0,0.0,0.0,0.0,211.289143,8.106,0,2,19,2010
3,1,2010-02-26,46.63,2.561,0.0,0.0,0.0,0.0,0.0,211.319643,8.106,0,2,26,2010
4,1,2010-03-05,46.50,2.625,0.0,0.0,0.0,0.0,0.0,211.350143,8.106,0,3,5,2010


In [0]:
features.describe

<bound method NDFrame.describe of       Store       Date  Temperature  Fuel_Price  ...  Tot_MDowns  Month  Day  Year
0         1 2010-02-05        42.31       2.572  ...        0.00      2    5  2010
1         1 2010-02-12        38.51       2.548  ...        0.00      2   12  2010
2         1 2010-02-19        39.93       2.514  ...        0.00      2   19  2010
3         1 2010-02-26        46.63       2.561  ...        0.00      2   26  2010
4         1 2010-03-05        46.50       2.625  ...        0.00      3    5  2010
...     ...        ...          ...         ...  ...         ...    ...  ...   ...
8185     45 2013-06-28        76.05       3.639  ...    11439.98      6   28  2013
8186     45 2013-07-05        77.50       3.614  ...    19254.20      7    5  2013
8187     45 2013-07-12        79.37       3.614  ...     8598.17      7   12  2013
8188     45 2013-07-19        82.84       3.737  ...     5635.21      7   19  2013
8189     45 2013-07-26        76.06       3.804  ... 

In [0]:
features.count()

Store           8190
Date            8190
Temperature     8190
Fuel_Price      8190
MarkDown1       8190
MarkDown2       8190
MarkDown3       8190
MarkDown4       8190
MarkDown5       8190
CPI             7605
Unemployment    7605
IsHoliday       8190
Tot_MDowns      8190
Month           8190
Day             8190
Year            8190
dtype: int64

Merge Stores

In [0]:
feats_and_stores = pd.merge(features,stores, on=['Store'], how='inner')

In [0]:
feats_and_stores.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Month,Day,Year,Type,Size
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,0,2,5,2010,A,151315
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,1,2,12,2010,A,151315
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,0,2,19,2010,A,151315
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,0,2,26,2010,A,151315
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,0,3,5,2010,A,151315


Group train data by week

In [0]:
train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


Fix dates

In [0]:
train['Date'] = pd.to_datetime(train['Date'])

In [0]:
train = train.groupby(['Store','Date','IsHoliday'])['Weekly_Sales'].sum()

In [0]:
train

Store  Date        IsHoliday
1      2010-02-05  False        1643690.90
       2010-02-12  True         1641957.44
       2010-02-19  False        1611968.17
       2010-02-26  False        1409727.59
       2010-03-05  False        1554806.68
                                   ...    
45     2012-09-28  False         713173.95
       2012-10-05  False         733455.07
       2012-10-12  False         734464.36
       2012-10-19  False         718125.53
       2012-10-26  False         760281.43
Name: Weekly_Sales, Length: 6435, dtype: float64

Merge data

In [0]:
feats_stores_train = pd.merge(feats_and_stores,train, on=['Store','Date'], how='inner')

In [0]:
feats_stores_train.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Month,Day,Year,Type,Size,Weekly_Sales
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,0,2,5,2010,A,151315,1643690.90
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,1,2,12,2010,A,151315,1641957.44
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,0,2,19,2010,A,151315,1611968.17
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,0,2,26,2010,A,151315,1409727.59
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,0,3,5,2010,A,151315,1554806.68


Eliminate past dates until Nov 4 2011...No markdown data at all

In [0]:
feats_stores_train[feats_stores_train['Date']>'2011-11-04']

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Month,Day,Year,Type,Size,Weekly_Sales
92,1,2011-11-11,59.11,3.297,10382.90,6115.67,215.07,2406.62,6551.42,217.998085,7.866,0,11,11,2011,A,151315,1594938.89
93,1,2011-11-18,62.25,3.308,6074.12,254.39,51.98,427.39,5988.57,218.220509,7.866,0,11,18,2011,A,151315,1539483.70
94,1,2011-11-25,60.14,3.236,410.31,98.00,55805.51,8.00,554.92,218.467621,7.866,1,11,25,2011,A,151315,2033320.66
95,1,2011-12-02,48.91,3.172,5629.51,68.00,1398.11,2084.64,20475.32,218.714733,7.866,0,12,2,2011,A,151315,1584083.95
96,1,2011-12-09,43.93,3.158,4640.65,19.00,105.02,3639.42,14461.82,218.961846,7.866,0,12,9,2011,A,151315,1799682.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6430,45,2012-09-28,64.88,3.997,4556.61,20.64,1.50,1601.01,3288.25,192.013558,8.684,0,9,28,2012,B,118221,713173.95
6431,45,2012-10-05,64.89,3.985,5046.74,NaN,18.82,2253.43,2340.01,192.170412,8.667,0,10,5,2012,B,118221,733455.07
6432,45,2012-10-12,54.47,4.000,1956.28,NaN,7.89,599.32,3990.54,192.327265,8.667,0,10,12,2012,B,118221,734464.36
6433,45,2012-10-19,56.47,3.969,2004.02,NaN,3.18,437.73,1537.49,192.330854,8.667,0,10,19,2012,B,118221,718125.53


Fill NAs

Fill Markdown NAs

In [0]:
feats_stores_train['MarkDown1'] = feats_stores_train['MarkDown1'].fillna(0)
feats_stores_train['MarkDown2'] = feats_stores_train['MarkDown2'].fillna(0)
feats_stores_train['MarkDown3'] = feats_stores_train['MarkDown3'].fillna(0)
feats_stores_train['MarkDown4'] = feats_stores_train['MarkDown4'].fillna(0)
feats_stores_train['MarkDown5'] = feats_stores_train['MarkDown5'].fillna(0)

Adding up Markdown Columns

In [0]:
feats_stores_train['Tot_MDowns'] = feats_stores_train['MarkDown1'] + feats_stores_train['MarkDown2'] + feats_stores_train['MarkDown3'] + feats_stores_train['MarkDown4'] + feats_stores_train['MarkDown5']

In [0]:
feats_stores_train.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Month,Day,Year,Type,Size,Weekly_Sales,Tot_MDowns
0,1,2010-02-05,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,0,2,5,2010,A,151315,1643690.90,0.0
1,1,2010-02-12,38.51,2.548,0.0,0.0,0.0,0.0,0.0,211.242170,8.106,1,2,12,2010,A,151315,1641957.44,0.0
2,1,2010-02-19,39.93,2.514,0.0,0.0,0.0,0.0,0.0,211.289143,8.106,0,2,19,2010,A,151315,1611968.17,0.0
3,1,2010-02-26,46.63,2.561,0.0,0.0,0.0,0.0,0.0,211.319643,8.106,0,2,26,2010,A,151315,1409727.59,0.0
4,1,2010-03-05,46.50,2.625,0.0,0.0,0.0,0.0,0.0,211.350143,8.106,0,3,5,2010,A,151315,1554806.68,0.0


In [0]:
dataset = feats_stores_train[['Weekly_Sales','Tot_MDowns','Store','Type','Size','Date','Year','Month','Day','Temperature','Fuel_Price','CPI','Unemployment','IsHoliday']]

In [0]:
dataset.tail()

,Weekly_Sales,Tot_MDowns,Store,Type,Size,Date,Year,Month,Day,Temperature,Fuel_Price,CPI,Unemployment,IsHoliday
6430,713173.95,9468.01,45,B,118221,2012-09-28,2012,9,28,64.88,3.997,192.013558,8.684,0
6431,733455.07,9659.00,45,B,118221,2012-10-05,2012,10,5,64.89,3.985,192.170412,8.667,0
6432,734464.36,6554.03,45,B,118221,2012-10-12,2012,10,12,54.47,4.000,192.327265,8.667,0
6433,718125.53,3982.42,45,B,118221,2012-10-19,2012,10,19,56.47,3.969,192.330854,8.667,0
6434,760281.43,5247.26,45,B,118221,2012-10-26,2012,10,26,58.85,3.882,192.308899,8.667,0
